In [1]:
import numpy as np
np.set_printoptions(precision=3, suppress=True)
import pandas as pd
import tensorflow as tf
import tensorflow_addons as tfa
import matplotlib as mpl
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.model_selection import KFold
import keras_tuner as kt
from sklearn.preprocessing import StandardScaler

2022-02-26 20:04:05.848819: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
### Load input data ###
data = pd.read_pickle('data_complete_final')
data.dropna(axis = 1, inplace = True)
data = data.drop(columns = [
    'rad_name', 'h_name', 'hash_u1', 'hash_u2', 'e_max_key', 'e_max', 'e_00', 'e_03', 'e_04',
    'e_05', 'e_06', 'e_07', 'e_10', 'meta_path', 'dG', 'rad_name_s',
    'h_name_s', 'reaction', 'trans_mean', 'Ea_trans_mean', 'rad_ref',
    'rad_ref_idx', 'h_ref', 'h_ref_idx', 'bur_vol_default_rad', 'bur_vol_2A_rad',
    'bur_vol_default_H', 'bur_vol_2A_H'
], axis = 1)

In [3]:
# compute enthalpies of reaction to use as a direct input
BDE_rad = data['rad_BDE']
BDE_H = data['H_BDE']
Delta_H = BDE_H - BDE_rad
Delta_H_df = pd.DataFrame(Delta_H, columns=['Delta_H'])

# Function Definitions

In [4]:
def unpack_column(dataframe_in, column):
    dataframe = dataframe_in.reset_index(drop=True)

    if len(dataframe[column][0].shape) > 1:
        max_length = max([len(row[0]) for row in dataframe[column]])
    else:
        max_length = max([len(row) for row in dataframe[column]])

    unpacked = np.empty((len(dataframe[column]), max_length))

    for i, row in zip(range(len(dataframe[column])), dataframe[column]):
        if len(row.shape) > 1:
            row = row[0]

        length_diff = max_length-len(row)
        if length_diff > 0:
            diff = np.zeros(max_length-len(row))
            expanded = np.concatenate((row, diff), axis=1)
            unpacked[i] = expanded
        else:
            unpacked[i] = row

    return pd.DataFrame(unpacked)

def drop_zeros(dataframe):
    df = dataframe.loc[:, (dataframe != 0).any(axis = 0)]
    return df
    
def unpack_join(dataframe, columns):
    unpacked_dfs = []
    for column in columns:
        unpacked_dfs.append(unpack_column(dataframe, column))

    dataframe_compl = dataframe.reset_index(drop=True)
    dataframe_compl.drop(columns = columns, inplace=True)
    for i, df in zip(range(len(unpacked_dfs)), unpacked_dfs):
        dataframe_compl = dataframe_compl.join(df, lsuffix = '_{}'.format(i-1) , rsuffix = '_{}'.format(i))
    
    return dataframe_compl  

def plot_loss(history, no_epochs, title):
    fs = 18
    fig, ax = plt.subplots(figsize = (10,10))
    ax.plot(history.history['loss'], label='Loss')
    ax.plot(history.history['val_loss'], label='Validation loss')
    ax.set_xlim([0, no_epochs])
    ax.set_xlabel('Epoch', fontsize = fs)
    ax.set_ylabel('Error [kcal/mol]', fontsize = fs)
    ax.tick_params(labelsize = fs)
    plt.legend(fontsize = fs)
    plt.grid(True)
    plt.title(title, fontsize = fs)
    plt.show()

def plot_predictions(model, test_features, test_labels, title):
    fs = 18
    fig, ax = plt.subplots(figsize = (10,10))
    test_predictions = model.predict(test_features).flatten()
    ax.scatter(test_predictions, test_labels)
    ax.set_xlabel('Predicted Barrier [kcal/mol]', fontsize = fs)
    ax.set_ylabel('True Barrier [kcal/mol]', fontsize = fs)
    lims = [0, max(test_labels)+10]
    ax.set_xlim(lims)
    ax.set_ylim(lims)
    ax.plot(lims, lims)
    ax.tick_params(labelsize = fs)
    plt.title(title, fontsize = fs)
    plt.show()

def plot_errors(model, test_features, test_labels, title):
    fs = 18
    fig, ax = plt.subplots(figsize = (10,10))
    test_predictions = model.predict(test_features).flatten()
    error = test_predictions - test_labels
    ax.hist(error, bins=5)
    ax.set_xlabel('Prediction error [kcal/mol]', fontsize = fs)
    ax.set_ylabel('Count', fontsize = fs)
    lims = [0, max(test_labels)+10]
    ax.set_xlim(lims)
    ax.set_ylim(lims)
    ax.plot(lims, lims)
    ax.tick_params(labelsize = fs)
    plt.title(title, fontsize = fs)
    plt.show()

def K_fold_cross_validation(
  inputs, targets, num_folds, model, mixed_dtypes, loss='MSLE', metrics = ['mae'], batch_size = 100, no_epochs = 500, verbose = 0,
  split = 0.2, callbacks=tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)
):
    mae_per_fold = []
    loss_per_fold = []
  
    lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
        0.001, decay_steps=inputs.shape[0]*0.8/batch_size*1000, decay_rate=1, staircase=False
    )
    optimizer=tf.keras.optimizers.Adam(lr_schedule)
  
  
    kfold = KFold(n_splits=num_folds, shuffle=True, random_state = 1)
    
    # K-fold Cross Validation model evaluation
    fold_no = 1
    for train, test in kfold.split(inputs, targets):
  
        model.compile(loss=loss,
                      optimizer=optimizer,
                      metrics=metrics)
                    
        print('------------------------------------------------------------------------')
        print(f'Training for fold {fold_no} ...')

        if mixed_dtypes:
            # inputs will be a DataFrame
            input_train_dict = {}
            input_train_dict = {name: np.array(value) 
                         for name, value in inputs.iloc[train,:].items()}
    
            history = model.fit(input_train_dict, targets[train],
                        batch_size=batch_size,
                        epochs=no_epochs,
                        verbose=verbose,
                        validation_split=split,
                        callbacks=callbacks)

            input_test_dict = {}
            input_test_dict = {name: np.array(value) 
                         for name, value in inputs.iloc[test,:].items()}

            scores = model.evaluate(input_test_dict, targets[test], verbose=0)

        else:
            # inputs will be a np.array
            history = model.fit(inputs[train], targets[train],
                        batch_size=batch_size,
                        epochs=no_epochs,
                        verbose=verbose,
                        validation_split=split,
                        callbacks=callbacks)
            scores = model.evaluate(inputs[test], targets[test], verbose=0)
        
        print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]}')
        mae_per_fold.append(scores[1])
        loss_per_fold.append(scores[0])
      
        fold_no = fold_no + 1
    
    print('------------------------------------------------------------------------')
    print('Score per fold')
    for i in range(0, len(mae_per_fold)):
        print('------------------------------------------------------------------------')
        print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - MAE: {mae_per_fold[i]}')
    print('------------------------------------------------------------------------')
    print('Average scores for all folds:')
    print(f'> MAE: {np.mean(mae_per_fold)} (+- {np.std(mae_per_fold)})')
    print(f'> Loss: {np.mean(loss_per_fold)}')
    print('------------------------------------------------------------------------')
  
    return mae_per_fold, loss_per_fold

def plot_cross_val_scores(dataframe):
    sns.set_style('ticks')
    ax = sns.violinplot(x="Model", y="MAE [kcal/mol]", data=dataframe, inner=None, orient='v')
    ax = sns.swarmplot(x="Model", y="MAE [kcal/mol]", data=dataframe, color='black', edgecolor='black')
    # sns.set(font_scale=1.1)

def plot_nn(names, shapes, activations, file_name, title):
    lengths = [len(names), len(shapes), len(activations)]
    lengths_comp = [len(names) for i in range(3)]
    assert np.allclose(lengths, lengths_comp), 'All input lists must be of the same length.'

    dot = Digraph(
        node_attr={'shape': 'box', 'fillcolor': 'lightblue2', 'style': 'filled'},
    )
    dot.attr(label=title)
    for i, n, s, a in zip(range(len(names)), names, shapes, activations):
        if s != None:
            dot.node('L{}'.format(i), '{}\n{} nodes\nActivation: {}'.format(n, s, a))
        else:
            dot.node('L{}'.format(i), '{}'.format(n))

    for i, j in zip(range(len(names)-1), range(1,len(names))):
        dot.edge('L{}'.format(i), 'L{}'.format(j))
        
    dot.format = 'png'
    dot.render(file_name)

def build_standard_model(input_shape):
    input = tf.keras.layers.Input(shape=(input_shape), dtype='float32')
    out = tf.keras.layers.Dense(128, activation='relu')(input)
    out = tf.keras.layers.BatchNormalization()(out)
    out = tf.keras.layers.Dense(64, activation='relu')(out)
    out = tf.keras.layers.BatchNormalization()(out)
    out = tf.keras.layers.Dense(1)(out)
    model = tf.keras.Model(input, out)
    return model

# Input data

In [5]:
data_soap_se_nomordred = data[[
    'Ea', 'translation', 'rad_charge', 'h_charge', 'rad_BDE', 'H_BDE',
    'max_spin_rad', 'mull_charge_rad', 'bur_vol_iso_rad',
    'max_spin_H', 'mull_charge_H', 'bur_vol_iso_H',
    'soap_H_start', 'soap_H_end'
]]

data_soap_se_noBDEs = data[[
    'Ea', 'translation', 'rad_charge', 'h_charge',
    'max_spin_rad', 'mull_charge_rad', 'bur_vol_iso_rad',
    'max_spin_H', 'mull_charge_H', 'bur_vol_iso_H',
    'nBase_rad', 'SpMax_A_rad', 'ATSC2s_rad', 'ATSC1Z_rad', 'ATSC2i_rad',
    'NdNH_rad', 'SMR_VSA4_rad', 'nBase_H', 'SpMax_A_H', 'ATSC2s_H',
    'ATSC1Z_H', 'ATSC2i_H', 'GATS2dv_H', 'BCUTdv-1h_H', 'SMR_VSA4_H',
    'VSA_EState7_H', 'soap_H_start', 'soap_H_end'
]]

data_soap_DeltaH = data[[
    'Ea', 'translation', 'rad_charge', 'h_charge',
    'max_spin_rad', 'mull_charge_rad', 'bur_vol_iso_rad',
    'max_spin_H', 'mull_charge_H', 'bur_vol_iso_H',
    'nBase_rad', 'SpMax_A_rad', 'ATSC2s_rad', 'ATSC1Z_rad', 'ATSC2i_rad',
    'NdNH_rad', 'SMR_VSA4_rad', 'nBase_H', 'SpMax_A_H', 'ATSC2s_H',
    'ATSC1Z_H', 'ATSC2i_H', 'GATS2dv_H', 'BCUTdv-1h_H', 'SMR_VSA4_H',
    'VSA_EState7_H', 'soap_H_start', 'soap_H_end'
]]
data_soap_DeltaH = data_soap_DeltaH.join(Delta_H_df)

data_lmbtr_se_noBDEs = data[[
    'Ea', 'translation', 'rad_charge', 'h_charge',
    'max_spin_rad', 'mull_charge_rad', 'bur_vol_iso_rad',
    'max_spin_H', 'mull_charge_H', 'bur_vol_iso_H',
    'nBase_rad', 'SpMax_A_rad', 'ATSC2s_rad', 'ATSC1Z_rad', 'ATSC2i_rad',
    'NdNH_rad', 'SMR_VSA4_rad', 'nBase_H', 'SpMax_A_H', 'ATSC2s_H',
    'ATSC1Z_H', 'ATSC2i_H', 'GATS2dv_H', 'BCUTdv-1h_H', 'SMR_VSA4_H',
    'VSA_EState7_H', 'lmbtr_H_start', 'lmbtr_H_end'
]]

data_lmbtr_DeltaH = data[[
    'Ea', 'translation', 'rad_charge', 'h_charge',
    'max_spin_rad', 'mull_charge_rad', 'bur_vol_iso_rad',
    'max_spin_H', 'mull_charge_H', 'bur_vol_iso_H',
    'nBase_rad', 'SpMax_A_rad', 'ATSC2s_rad', 'ATSC1Z_rad', 'ATSC2i_rad',
    'NdNH_rad', 'SMR_VSA4_rad', 'nBase_H', 'SpMax_A_H', 'ATSC2s_H',
    'ATSC1Z_H', 'ATSC2i_H', 'GATS2dv_H', 'BCUTdv-1h_H', 'SMR_VSA4_H',
    'VSA_EState7_H', 'lmbtr_H_start', 'lmbtr_H_end'
]]
data_lmbtr_DeltaH = data_lmbtr_DeltaH.join(Delta_H_df)

In [6]:
data_soap_se_nomordred = unpack_join(data_soap_se_nomordred, ['soap_H_start', 'soap_H_end'])
data_soap_se_nomordred = drop_zeros(data_soap_se_nomordred)

data_soap_se_noBDEs = unpack_join(data_soap_se_noBDEs, ['soap_H_start', 'soap_H_end'])
data_soap_se_noBDEs = drop_zeros(data_soap_se_noBDEs)

data_soap_DeltaH = unpack_join(data_soap_DeltaH, ['soap_H_start', 'soap_H_end'])
data_soap_DeltaH = drop_zeros(data_soap_DeltaH)

data_lmbtr_se_noBDEs = unpack_join(data_lmbtr_se_noBDEs, ['lmbtr_H_start', 'lmbtr_H_end'])
data_lmbtr_se_noBDEs = drop_zeros(data_lmbtr_se_noBDEs)

data_lmbtr_DeltaH = unpack_join(data_lmbtr_DeltaH, ['lmbtr_H_start', 'lmbtr_H_end'])
data_lmbtr_DeltaH = drop_zeros(data_lmbtr_DeltaH)

In [7]:
print(
    '''
    Shape of inputs:\n
    data_soap_se_nomordred: {}\n
    data_soap_se_noBDEs: {}\n
    data_soap_DeltaH: {}\n
    data_lmbtr_se_noBDEs: {}\n
    data_lmbtr_DeltaH: {}
    '''.format(
        data_soap_se_nomordred.shape[1],
        data_soap_se_noBDEs.shape[1],
        data_soap_DeltaH.shape[1],
        data_lmbtr_se_noBDEs.shape[1],
        data_lmbtr_DeltaH.shape[1]
    )
)


    Shape of inputs:

    data_soap_se_nomordred: 11492

    data_soap_se_noBDEs: 11506

    data_soap_DeltaH: 11507

    data_lmbtr_se_noBDEs: 8240

    data_lmbtr_DeltaH: 8241
    


# Introduce Normalization

In [9]:
# SOAP descriptors, Mordred descriptors
targets = data_soap_se_nomordred.pop('Ea')
inputs = np.array(data_soap_se_nomordred)

normalize = tf.keras.layers.Normalization()
normalize.adapt(inputs)

input = tf.keras.layers.Input(shape=(11491), dtype='float32')
out = normalize(input)
out = tf.keras.layers.Dense(128, activation='relu')(out)
out = tf.keras.layers.BatchNormalization()(out)
out = tf.keras.layers.Dense(64, activation='relu')(out)
out = tf.keras.layers.BatchNormalization()(out)
out = tf.keras.layers.Dense(1)(out)
model = tf.keras.Model(input, out)

mae_per_fold_soap_se_nomordred, loss_per_fold_soap_se_nomordred = K_fold_cross_validation(
    inputs, targets, 10, model, mixed_dtypes=False, loss='mae', no_epochs=30, batch_size=32
)

------------------------------------------------------------------------
Training for fold 1 ...
Score for fold 1: loss of 8.013965606689453; mae of 8.013965606689453
------------------------------------------------------------------------
Training for fold 2 ...
Score for fold 2: loss of 5.976191997528076; mae of 5.976191997528076
------------------------------------------------------------------------
Training for fold 3 ...
Score for fold 3: loss of 5.625258922576904; mae of 5.625258922576904
------------------------------------------------------------------------
Training for fold 4 ...
Score for fold 4: loss of 4.338718891143799; mae of 4.338718891143799
------------------------------------------------------------------------
Training for fold 5 ...
Score for fold 5: loss of 4.963077545166016; mae of 4.963077545166016
------------------------------------------------------------------------
Training for fold 6 ...
Score for fold 6: loss of 4.5812811851501465; mae of 4.5812811851501

In [13]:
# SOAP descriptors, no BDEs
targets = data_soap_se_noBDEs.pop('Ea')
inputs = np.array(data_soap_se_noBDEs)

normalize = tf.keras.layers.Normalization()
normalize.adapt(inputs)

input = tf.keras.layers.Input(shape=(11505), dtype='float32')
out = normalize(input)
out = tf.keras.layers.Dense(128, activation='relu')(out)
out = tf.keras.layers.BatchNormalization()(out)
out = tf.keras.layers.Dense(64, activation='relu')(out)
out = tf.keras.layers.BatchNormalization()(out)
out = tf.keras.layers.Dense(1)(out)
model = tf.keras.Model(input, out)

mae_per_fold_soap_se_noBDEs, loss_per_fold_soap_se_noBDEs = K_fold_cross_validation(
    inputs, targets, 10, model, mixed_dtypes=False, loss='mae', no_epochs=30, batch_size=32
)

------------------------------------------------------------------------
Training for fold 1 ...
Score for fold 1: loss of 8.202195167541504; mae of 8.202195167541504
------------------------------------------------------------------------
Training for fold 2 ...
Score for fold 2: loss of 6.427987575531006; mae of 6.427987575531006
------------------------------------------------------------------------
Training for fold 3 ...
Score for fold 3: loss of 5.738282203674316; mae of 5.738282203674316
------------------------------------------------------------------------
Training for fold 4 ...
Score for fold 4: loss of 4.562415599822998; mae of 4.562415599822998
------------------------------------------------------------------------
Training for fold 5 ...
Score for fold 5: loss of 5.016743183135986; mae of 5.016743183135986
------------------------------------------------------------------------
Training for fold 6 ...
Score for fold 6: loss of 4.356122016906738; mae of 4.35612201690673

In [14]:
# SOAP descriptors, Mordred descriptors, enthalpy of reaction
targets = data_soap_DeltaH.pop('Ea')
inputs = np.array(data_soap_DeltaH)

normalize = tf.keras.layers.Normalization()
normalize.adapt(inputs)

input = tf.keras.layers.Input(shape=(11506), dtype='float32')
out = normalize(input)
out = tf.keras.layers.Dense(128, activation='relu')(out)
out = tf.keras.layers.BatchNormalization()(out)
out = tf.keras.layers.Dense(64, activation='relu')(out)
out = tf.keras.layers.BatchNormalization()(out)
out = tf.keras.layers.Dense(1)(out)
model = tf.keras.Model(input, out)

mae_per_fold_soap_DeltaH, loss_per_fold_soap_DeltaH = K_fold_cross_validation(
    inputs, targets, 10, model, mixed_dtypes=False, loss='mae', no_epochs=30, batch_size=32
)

------------------------------------------------------------------------
Training for fold 1 ...
Score for fold 1: loss of 8.333030700683594; mae of 8.333030700683594
------------------------------------------------------------------------
Training for fold 2 ...
Score for fold 2: loss of 6.098076820373535; mae of 6.098076820373535
------------------------------------------------------------------------
Training for fold 3 ...
Score for fold 3: loss of 5.539495944976807; mae of 5.539495944976807
------------------------------------------------------------------------
Training for fold 4 ...
Score for fold 4: loss of 4.559484481811523; mae of 4.559484481811523
------------------------------------------------------------------------
Training for fold 5 ...
Score for fold 5: loss of 4.750124454498291; mae of 4.750124454498291
------------------------------------------------------------------------
Training for fold 6 ...
Score for fold 6: loss of 4.2207231521606445; mae of 4.2207231521606

In [15]:
# LMBTR descriptors, Mordred descriptors
targets = data_lmbtr_se_noBDEs.pop('Ea')
inputs = np.array(data_lmbtr_se_noBDEs)

normalize = tf.keras.layers.Normalization()
normalize.adapt(inputs)

input = tf.keras.layers.Input(shape=(8239), dtype='float32')
out = normalize(input)
out = tf.keras.layers.Dense(128, activation='relu')(out)
out = tf.keras.layers.BatchNormalization()(out)
out = tf.keras.layers.Dense(64, activation='relu')(out)
out = tf.keras.layers.BatchNormalization()(out)
out = tf.keras.layers.Dense(1)(out)
model = tf.keras.Model(input, out)

mae_per_fold_lmbtr_se_noBDEs, loss_per_fold_lmbtr_se_noBDEs = K_fold_cross_validation(
    inputs, targets, 10, model, mixed_dtypes=False, loss='mae', no_epochs=30, batch_size=32
)

------------------------------------------------------------------------
Training for fold 1 ...
Score for fold 1: loss of 8.266844749450684; mae of 8.266844749450684
------------------------------------------------------------------------
Training for fold 2 ...
Score for fold 2: loss of 5.354099750518799; mae of 5.354099750518799
------------------------------------------------------------------------
Training for fold 3 ...
Score for fold 3: loss of 4.664579391479492; mae of 4.664579391479492
------------------------------------------------------------------------
Training for fold 4 ...
Score for fold 4: loss of 3.975088596343994; mae of 3.975088596343994
------------------------------------------------------------------------
Training for fold 5 ...
Score for fold 5: loss of 4.582648277282715; mae of 4.582648277282715
------------------------------------------------------------------------
Training for fold 6 ...
Score for fold 6: loss of 3.981537342071533; mae of 3.98153734207153

In [16]:
# LMBTR descriptors, Mordred descriptors, enthalpy of reaction
targets = data_lmbtr_DeltaH.pop('Ea')
inputs = np.array(data_lmbtr_DeltaH)

normalize = tf.keras.layers.Normalization()
normalize.adapt(inputs)

input = tf.keras.layers.Input(shape=(8240), dtype='float32')
out = normalize(input)
out = tf.keras.layers.Dense(128, activation='relu')(out)
out = tf.keras.layers.BatchNormalization()(out)
out = tf.keras.layers.Dense(64, activation='relu')(out)
out = tf.keras.layers.BatchNormalization()(out)
out = tf.keras.layers.Dense(1)(out)
model = tf.keras.Model(input, out)

mae_per_fold_lmbtr_DeltaH, loss_per_fold_lmbtr_DeltaH = K_fold_cross_validation(
    inputs, targets, 10, model, mixed_dtypes=False, loss='mae', no_epochs=30, batch_size=32
)

------------------------------------------------------------------------
Training for fold 1 ...
Score for fold 1: loss of 8.305031776428223; mae of 8.305031776428223
------------------------------------------------------------------------
Training for fold 2 ...
Score for fold 2: loss of 5.074409008026123; mae of 5.074409008026123
------------------------------------------------------------------------
Training for fold 3 ...
Score for fold 3: loss of 4.699100971221924; mae of 4.699100971221924
------------------------------------------------------------------------
Training for fold 4 ...
Score for fold 4: loss of 4.084972858428955; mae of 4.084972858428955
------------------------------------------------------------------------
Training for fold 5 ...
Score for fold 5: loss of 4.359579563140869; mae of 4.359579563140869
------------------------------------------------------------------------
Training for fold 6 ...
Score for fold 6: loss of 4.083934307098389; mae of 4.08393430709838

In [17]:
df_hyper_opt_norm = pd.DataFrame({'Model': [
    *['Model 1' for i in range(len(mae_per_fold_soap_se_nomordred))],
    *['Model 2' for i in range(len(mae_per_fold_soap_se_noBDEs))],
    *['Model 3' for i in range(len(mae_per_fold_soap_DeltaH))],
    *['Model 4' for i in range(len(mae_per_fold_lmbtr_se_noBDEs))],
    *['Model 5' for i in range(len(mae_per_fold_lmbtr_DeltaH))]
], 'MAE [kcal/mol]': [
    *mae_per_fold_soap_se_nomordred, *mae_per_fold_soap_se_noBDEs, *mae_per_fold_soap_DeltaH,
    *mae_per_fold_lmbtr_se_noBDEs, *mae_per_fold_lmbtr_DeltaH
]})

df_hyper_opt_norm.to_csv('cross_val_scores_hyper_opt_norm.csv')

The implementation of normalization above is incompatible with the tensorflow version required for GPU acceleration, so for further hyperparameter optimization we'll normalize the date before we pass it to the input layer.

In [7]:
# reload the data first
data_soap_se_nomordred_target = data_soap_se_nomordred.pop('Ea')
data_soap_se_noBDEs_target = data_soap_se_noBDEs.pop('Ea')
data_soap_DeltaH_target = data_soap_DeltaH.pop('Ea')
data_lmbtr_se_noBDEs_target = data_lmbtr_se_noBDEs.pop('Ea')
data_lmbtr_DeltaH_target = data_lmbtr_DeltaH.pop('Ea')

data_soap_se_nomordred = pd.DataFrame(StandardScaler().fit_transform(data_soap_se_nomordred), columns = list(data_soap_se_nomordred.columns))
data_soap_se_noBDEs = pd.DataFrame(StandardScaler().fit_transform(data_soap_se_noBDEs), columns = list(data_soap_se_noBDEs.columns))
data_soap_DeltaH = pd.DataFrame(StandardScaler().fit_transform(data_soap_DeltaH), columns = list(data_soap_DeltaH.columns))
data_lmbtr_se_noBDEs = pd.DataFrame(StandardScaler().fit_transform(data_lmbtr_se_noBDEs), columns = list(data_lmbtr_se_noBDEs.columns))
data_lmbtr_DeltaH = pd.DataFrame(StandardScaler().fit_transform(data_lmbtr_DeltaH), columns = list(data_lmbtr_DeltaH.columns))

# Add layers (up to 8)

In [8]:
def build_model_2(input_shape):
    input = tf.keras.layers.Input(shape=(input_shape), dtype='float32')
    out = tf.keras.layers.Dense(128, activation='relu')(input)
    out = tf.keras.layers.BatchNormalization()(out)
    out = tf.keras.layers.Dense(128, activation='relu')(out)
    out = tf.keras.layers.BatchNormalization()(out)
    out = tf.keras.layers.Dense(64, activation='relu')(out)
    out = tf.keras.layers.BatchNormalization()(out)
    out = tf.keras.layers.Dense(1)(out)
    model = tf.keras.Model(input, out)
    return model

def build_model_3(input_shape):
    input = tf.keras.layers.Input(shape=(input_shape), dtype='float32')
    out = tf.keras.layers.Dense(128, activation='relu')(input)
    out = tf.keras.layers.BatchNormalization()(out)
    out = tf.keras.layers.Dense(128, activation='relu')(out)
    out = tf.keras.layers.BatchNormalization()(out)
    out = tf.keras.layers.Dense(128, activation='relu')(out)
    out = tf.keras.layers.BatchNormalization()(out)
    out = tf.keras.layers.Dense(64, activation='relu')(out)
    out = tf.keras.layers.BatchNormalization()(out)
    out = tf.keras.layers.Dense(1)(out)
    model = tf.keras.Model(input, out)
    return model

def build_model_4(input_shape):
    input = tf.keras.layers.Input(shape=(input_shape), dtype='float32')
    out = tf.keras.layers.Dense(128, activation='relu')(input)
    out = tf.keras.layers.BatchNormalization()(out)
    out = tf.keras.layers.Dense(128, activation='relu')(out)
    out = tf.keras.layers.BatchNormalization()(out)
    out = tf.keras.layers.Dense(128, activation='relu')(out)
    out = tf.keras.layers.BatchNormalization()(out)
    out = tf.keras.layers.Dense(128, activation='relu')(out)
    out = tf.keras.layers.BatchNormalization()(out)
    out = tf.keras.layers.Dense(64, activation='relu')(out)
    out = tf.keras.layers.BatchNormalization()(out)
    out = tf.keras.layers.Dense(1)(out)
    model = tf.keras.Model(input, out)
    return model

def build_model_5(input_shape):
    input = tf.keras.layers.Input(shape=(input_shape), dtype='float32')
    out = tf.keras.layers.Dense(128, activation='relu')(input)
    out = tf.keras.layers.BatchNormalization()(out)
    out = tf.keras.layers.Dense(128, activation='relu')(out)
    out = tf.keras.layers.BatchNormalization()(out)
    out = tf.keras.layers.Dense(128, activation='relu')(out)
    out = tf.keras.layers.BatchNormalization()(out)
    out = tf.keras.layers.Dense(128, activation='relu')(out)
    out = tf.keras.layers.BatchNormalization()(out)
    out = tf.keras.layers.Dense(128, activation='relu')(out)
    out = tf.keras.layers.BatchNormalization()(out)
    out = tf.keras.layers.Dense(64, activation='relu')(out)
    out = tf.keras.layers.BatchNormalization()(out)
    out = tf.keras.layers.Dense(1)(out)
    model = tf.keras.Model(input, out)
    return model

def build_model_6(input_shape):
    input = tf.keras.layers.Input(shape=(input_shape), dtype='float32')
    out = tf.keras.layers.Dense(128, activation='relu')(input)
    out = tf.keras.layers.BatchNormalization()(out)
    out = tf.keras.layers.Dense(128, activation='relu')(out)
    out = tf.keras.layers.BatchNormalization()(out)
    out = tf.keras.layers.Dense(128, activation='relu')(out)
    out = tf.keras.layers.BatchNormalization()(out)
    out = tf.keras.layers.Dense(128, activation='relu')(out)
    out = tf.keras.layers.BatchNormalization()(out)
    out = tf.keras.layers.Dense(128, activation='relu')(out)
    out = tf.keras.layers.BatchNormalization()(out)
    out = tf.keras.layers.Dense(128, activation='relu')(out)
    out = tf.keras.layers.BatchNormalization()(out)
    out = tf.keras.layers.Dense(64, activation='relu')(out)
    out = tf.keras.layers.BatchNormalization()(out)
    out = tf.keras.layers.Dense(1)(out)
    model = tf.keras.Model(input, out)
    return model

## 4 layers

In [10]:
# SOAP descriptors, Mordred descriptors
model = build_model_2(11491)
inputs = np.array(data_soap_se_nomordred)
mae_per_fold_soap_se_nomordred_2, loss_per_fold_soap_se_nomordred_2 = K_fold_cross_validation(
    inputs, data_soap_se_nomordred_target, 10, model, mixed_dtypes=False, loss='mae', no_epochs=30, batch_size=32
)

------------------------------------------------------------------------
Training for fold 1 ...
Score for fold 1: loss of 8.090597152709961; mae of 8.090597152709961
------------------------------------------------------------------------
Training for fold 2 ...
Score for fold 2: loss of 6.224880218505859; mae of 6.224880218505859
------------------------------------------------------------------------
Training for fold 3 ...
Score for fold 3: loss of 5.649747371673584; mae of 5.649747371673584
------------------------------------------------------------------------
Training for fold 4 ...
Score for fold 4: loss of 4.367082118988037; mae of 4.367082118988037
------------------------------------------------------------------------
Training for fold 5 ...
Score for fold 5: loss of 4.847179889678955; mae of 4.847179889678955
------------------------------------------------------------------------
Training for fold 6 ...
Score for fold 6: loss of 4.263080596923828; mae of 4.26308059692382

In [11]:
# SOAP descriptors, BDEs
model = build_model_2(11505)
inputs = np.array(data_soap_se_noBDEs)
mae_per_fold_soap_se_noBDEs_2, loss_per_fold_soap_se_noBDEs_2 = K_fold_cross_validation(
    inputs, data_soap_se_noBDEs_target, 10, model, mixed_dtypes=False, loss='mae', no_epochs=30, batch_size=32
)

------------------------------------------------------------------------
Training for fold 1 ...
Score for fold 1: loss of 8.046910285949707; mae of 8.046910285949707
------------------------------------------------------------------------
Training for fold 2 ...
Score for fold 2: loss of 6.037797927856445; mae of 6.037797927856445
------------------------------------------------------------------------
Training for fold 3 ...
Score for fold 3: loss of 5.384449481964111; mae of 5.384449481964111
------------------------------------------------------------------------
Training for fold 4 ...
Score for fold 4: loss of 4.563538074493408; mae of 4.563538074493408
------------------------------------------------------------------------
Training for fold 5 ...
Score for fold 5: loss of 4.916438102722168; mae of 4.916438102722168
------------------------------------------------------------------------
Training for fold 6 ...
Score for fold 6: loss of 4.1573405265808105; mae of 4.1573405265808

In [12]:
# SOAP descriptors, Mordred descriptors, enthalpy of reaction
model = build_model_2(11506)
inputs = np.array(data_soap_DeltaH)
mae_per_fold_soap_DeltaH_2, loss_per_fold_soap_DeltaH_2 = K_fold_cross_validation(
    inputs, data_soap_DeltaH_target, 10, model, mixed_dtypes=False, loss='mae', no_epochs=30, batch_size=32
)

------------------------------------------------------------------------
Training for fold 1 ...
Score for fold 1: loss of 8.091449737548828; mae of 8.091449737548828
------------------------------------------------------------------------
Training for fold 2 ...
Score for fold 2: loss of 5.927804470062256; mae of 5.927804470062256
------------------------------------------------------------------------
Training for fold 3 ...
Score for fold 3: loss of 5.8144917488098145; mae of 5.8144917488098145
------------------------------------------------------------------------
Training for fold 4 ...
Score for fold 4: loss of 4.698416709899902; mae of 4.698416709899902
------------------------------------------------------------------------
Training for fold 5 ...
Score for fold 5: loss of 4.906053066253662; mae of 4.906053066253662
------------------------------------------------------------------------
Training for fold 6 ...
Score for fold 6: loss of 4.4060378074646; mae of 4.4060378074646


In [13]:
# LMBTR descriptors, Mordred descriptors
model = build_model_2(8239)
inputs = np.array(data_lmbtr_se_noBDEs)
mae_per_fold_lmbtr_se_noBDEs_2, loss_per_fold_lmbtr_se_noBDEs_2 = K_fold_cross_validation(
    inputs, data_lmbtr_se_noBDEs_target, 10, model, mixed_dtypes=False, loss='mae', no_epochs=30, batch_size=32
)

------------------------------------------------------------------------
Training for fold 1 ...
Score for fold 1: loss of 8.181506156921387; mae of 8.181506156921387
------------------------------------------------------------------------
Training for fold 2 ...
Score for fold 2: loss of 5.480401039123535; mae of 5.480401039123535
------------------------------------------------------------------------
Training for fold 3 ...
Score for fold 3: loss of 4.351189136505127; mae of 4.351189136505127
------------------------------------------------------------------------
Training for fold 4 ...
Score for fold 4: loss of 3.9752748012542725; mae of 3.9752748012542725
------------------------------------------------------------------------
Training for fold 5 ...
Score for fold 5: loss of 4.494346618652344; mae of 4.494346618652344
------------------------------------------------------------------------
Training for fold 6 ...
Score for fold 6: loss of 4.374698162078857; mae of 4.374698162078

In [14]:
# LMBTR descriptors, Mordred descriptors, enthalpy of reaction
model = build_model_2(8240)
inputs = np.array(data_lmbtr_DeltaH)
mae_per_fold_lmbtr_DeltaH_2, loss_per_fold_lmbtr_DeltaH_2 = K_fold_cross_validation(
    inputs, data_lmbtr_DeltaH_target, 10, model, mixed_dtypes=False, loss='mae', no_epochs=30, batch_size=32
)

------------------------------------------------------------------------
Training for fold 1 ...
Score for fold 1: loss of 8.074642181396484; mae of 8.074642181396484
------------------------------------------------------------------------
Training for fold 2 ...
Score for fold 2: loss of 5.372731685638428; mae of 5.372731685638428
------------------------------------------------------------------------
Training for fold 3 ...
Score for fold 3: loss of 4.708434581756592; mae of 4.708434581756592
------------------------------------------------------------------------
Training for fold 4 ...
Score for fold 4: loss of 3.8841285705566406; mae of 3.8841285705566406
------------------------------------------------------------------------
Training for fold 5 ...
Score for fold 5: loss of 4.156886577606201; mae of 4.156886577606201
------------------------------------------------------------------------
Training for fold 6 ...
Score for fold 6: loss of 4.141538619995117; mae of 4.141538619995

In [15]:
df_hyper_opt_more_layers = pd.DataFrame({'Model': [
    *['Model 1 (4 layers)' for i in range(len(mae_per_fold_soap_se_nomordred_2))],
    *['Model 2 (4 layers)' for i in range(len(mae_per_fold_soap_se_noBDEs_2))],
    *['Model 3 (4 layers)' for i in range(len(mae_per_fold_soap_DeltaH_2))],
    *['Model 4 (4 layers)' for i in range(len(mae_per_fold_lmbtr_se_noBDEs_2))],
    *['Model 5 (4 layers)' for i in range(len(mae_per_fold_lmbtr_DeltaH_2))]
], 'MAE [kcal/mol]': [
    *mae_per_fold_soap_se_nomordred_2, *mae_per_fold_soap_se_noBDEs_2, *mae_per_fold_soap_DeltaH_2,
    *mae_per_fold_lmbtr_se_noBDEs_2, *mae_per_fold_lmbtr_DeltaH_2
]})

df_hyper_opt_more_layers.to_csv('cross_val_scores_hyper_opt_more_layers.csv')

## 5 layers

In [16]:
# SOAP descriptors, Mordred descriptors
model = build_model_3(11491)
inputs = np.array(data_soap_se_nomordred)
mae_per_fold_soap_se_nomordred_3, loss_per_fold_soap_se_nomordred_3 = K_fold_cross_validation(
    inputs, data_soap_se_nomordred_target, 10, model, mixed_dtypes=False, loss='mae', no_epochs=30, batch_size=32
)

------------------------------------------------------------------------
Training for fold 1 ...
Score for fold 1: loss of 7.725742340087891; mae of 7.725742340087891
------------------------------------------------------------------------
Training for fold 2 ...
Score for fold 2: loss of 6.308877468109131; mae of 6.308877468109131
------------------------------------------------------------------------
Training for fold 3 ...
Score for fold 3: loss of 5.510425567626953; mae of 5.510425567626953
------------------------------------------------------------------------
Training for fold 4 ...
Score for fold 4: loss of 4.984089374542236; mae of 4.984089374542236
------------------------------------------------------------------------
Training for fold 5 ...
Score for fold 5: loss of 4.95871639251709; mae of 4.95871639251709
------------------------------------------------------------------------
Training for fold 6 ...
Score for fold 6: loss of 4.3913140296936035; mae of 4.391314029693603

In [17]:
# SOAP descriptors, BDEs
model = build_model_3(11505)
inputs = np.array(data_soap_se_noBDEs)
mae_per_fold_soap_se_noBDEs_3, loss_per_fold_soap_se_noBDEs_3 = K_fold_cross_validation(
    inputs, data_soap_se_noBDEs_target, 10, model, mixed_dtypes=False, loss='mae', no_epochs=30, batch_size=32
)

------------------------------------------------------------------------
Training for fold 1 ...
Score for fold 1: loss of 7.769536018371582; mae of 7.769536018371582
------------------------------------------------------------------------
Training for fold 2 ...
Score for fold 2: loss of 6.313778877258301; mae of 6.313778877258301
------------------------------------------------------------------------
Training for fold 3 ...
Score for fold 3: loss of 5.504204273223877; mae of 5.504204273223877
------------------------------------------------------------------------
Training for fold 4 ...
Score for fold 4: loss of 4.686505317687988; mae of 4.686505317687988
------------------------------------------------------------------------
Training for fold 5 ...
Score for fold 5: loss of 4.909460544586182; mae of 4.909460544586182
------------------------------------------------------------------------
Training for fold 6 ...
Score for fold 6: loss of 4.7480549812316895; mae of 4.7480549812316

In [18]:
# SOAP descriptors, Mordred descriptors, enthalpy of reaction
model = build_model_3(11506)
inputs = np.array(data_soap_DeltaH)
mae_per_fold_soap_DeltaH_3, loss_per_fold_soap_DeltaH_3 = K_fold_cross_validation(
    inputs, data_soap_DeltaH_target, 10, model, mixed_dtypes=False, loss='mae', no_epochs=30, batch_size=32
)

------------------------------------------------------------------------
Training for fold 1 ...
Score for fold 1: loss of 7.9345903396606445; mae of 7.9345903396606445
------------------------------------------------------------------------
Training for fold 2 ...
Score for fold 2: loss of 6.316888332366943; mae of 6.316888332366943
------------------------------------------------------------------------
Training for fold 3 ...
Score for fold 3: loss of 5.694052696228027; mae of 5.694052696228027
------------------------------------------------------------------------
Training for fold 4 ...
Score for fold 4: loss of 4.833024501800537; mae of 4.833024501800537
------------------------------------------------------------------------
Training for fold 5 ...
Score for fold 5: loss of 5.199148654937744; mae of 5.199148654937744
------------------------------------------------------------------------
Training for fold 6 ...
Score for fold 6: loss of 4.351123332977295; mae of 4.351123332977

In [19]:
# LMBTR descriptors, Mordred descriptors
model = build_model_3(8239)
inputs = np.array(data_lmbtr_se_noBDEs)
mae_per_fold_lmbtr_se_noBDEs_3, loss_per_fold_lmbtr_se_noBDEs_3 = K_fold_cross_validation(
    inputs, data_lmbtr_se_noBDEs_target, 10, model, mixed_dtypes=False, loss='mae', no_epochs=30, batch_size=32
)

------------------------------------------------------------------------
Training for fold 1 ...
Score for fold 1: loss of 8.214994430541992; mae of 8.214994430541992
------------------------------------------------------------------------
Training for fold 2 ...
Score for fold 2: loss of 5.251453876495361; mae of 5.251453876495361
------------------------------------------------------------------------
Training for fold 3 ...
Score for fold 3: loss of 4.436524868011475; mae of 4.436524868011475
------------------------------------------------------------------------
Training for fold 4 ...
Score for fold 4: loss of 4.394689559936523; mae of 4.394689559936523
------------------------------------------------------------------------
Training for fold 5 ...
Score for fold 5: loss of 4.332404613494873; mae of 4.332404613494873
------------------------------------------------------------------------
Training for fold 6 ...
Score for fold 6: loss of 4.203233242034912; mae of 4.20323324203491

In [20]:
# LMBTR descriptors, Mordred descriptors, enthalpy of reaction
model = build_model_3(8240)
inputs = np.array(data_lmbtr_DeltaH)
mae_per_fold_lmbtr_DeltaH_3, loss_per_fold_lmbtr_DeltaH_3 = K_fold_cross_validation(
    inputs, data_lmbtr_DeltaH_target, 10, model, mixed_dtypes=False, loss='mae', no_epochs=30, batch_size=32
)

------------------------------------------------------------------------
Training for fold 1 ...
Score for fold 1: loss of 8.687891006469727; mae of 8.687891006469727
------------------------------------------------------------------------
Training for fold 2 ...
Score for fold 2: loss of 5.695990562438965; mae of 5.695990562438965
------------------------------------------------------------------------
Training for fold 3 ...
Score for fold 3: loss of 4.3831071853637695; mae of 4.3831071853637695
------------------------------------------------------------------------
Training for fold 4 ...
Score for fold 4: loss of 4.209273338317871; mae of 4.209273338317871
------------------------------------------------------------------------
Training for fold 5 ...
Score for fold 5: loss of 4.383842468261719; mae of 4.383842468261719
------------------------------------------------------------------------
Training for fold 6 ...
Score for fold 6: loss of 4.087885856628418; mae of 4.087885856628

In [21]:
df_hyper_opt_more_layers = pd.read_csv('cross_val_scores_hyper_opt_more_layers.csv', index_col=0)

new_scores = pd.DataFrame({'Model': [
    *['Model 1 (5 layers)' for i in range(len(mae_per_fold_soap_se_nomordred_3))],
    *['Model 2 (5 layers)' for i in range(len(mae_per_fold_soap_se_noBDEs_3))],
    *['Model 3 (5 layers)' for i in range(len(mae_per_fold_soap_DeltaH_3))],
    *['Model 4 (5 layers)' for i in range(len(mae_per_fold_lmbtr_se_noBDEs_3))],
    *['Model 5 (5 layers)' for i in range(len(mae_per_fold_lmbtr_DeltaH_3))]
], 'MAE [kcal/mol]': [
    *mae_per_fold_soap_se_nomordred_3, *mae_per_fold_soap_se_noBDEs_3, *mae_per_fold_soap_DeltaH_3,
    *mae_per_fold_lmbtr_se_noBDEs_3, *mae_per_fold_lmbtr_DeltaH_3
]})
df_hyper_opt_more_layers = pd.concat([df_hyper_opt_more_layers, new_scores])
df_hyper_opt_more_layers.to_csv('cross_val_scores_hyper_opt_more_layers.csv')

## 6 layers

In [24]:
# SOAP descriptors, Mordred descriptors
model = build_model_4(11491)
inputs = np.array(data_soap_se_nomordred)
mae_per_fold_soap_se_nomordred_4, loss_per_fold_soap_se_nomordred_4 = K_fold_cross_validation(
    inputs, data_soap_se_nomordred_target, 10, model, mixed_dtypes=False, loss='mae', no_epochs=30, batch_size=32
)

------------------------------------------------------------------------
Training for fold 1 ...
Score for fold 1: loss of 8.072370529174805; mae of 8.072370529174805
------------------------------------------------------------------------
Training for fold 2 ...
Score for fold 2: loss of 6.388580799102783; mae of 6.388580799102783
------------------------------------------------------------------------
Training for fold 3 ...
Score for fold 3: loss of 5.644259452819824; mae of 5.644259452819824
------------------------------------------------------------------------
Training for fold 4 ...
Score for fold 4: loss of 5.043553829193115; mae of 5.043553829193115
------------------------------------------------------------------------
Training for fold 5 ...
Score for fold 5: loss of 4.987371921539307; mae of 4.987371921539307
------------------------------------------------------------------------
Training for fold 6 ...
Score for fold 6: loss of 4.252241134643555; mae of 4.25224113464355

In [25]:
# SOAP descriptors, BDEs
model = build_model_4(11505)
inputs = np.array(data_soap_se_noBDEs)
mae_per_fold_soap_se_noBDEs_4, loss_per_fold_soap_se_noBDEs_4 = K_fold_cross_validation(
    inputs, data_soap_se_noBDEs_target, 10, model, mixed_dtypes=False, loss='mae', no_epochs=30, batch_size=32
)

------------------------------------------------------------------------
Training for fold 1 ...
Score for fold 1: loss of 8.082093238830566; mae of 8.082093238830566
------------------------------------------------------------------------
Training for fold 2 ...
Score for fold 2: loss of 6.57771110534668; mae of 6.57771110534668
------------------------------------------------------------------------
Training for fold 3 ...
Score for fold 3: loss of 5.848302841186523; mae of 5.848302841186523
------------------------------------------------------------------------
Training for fold 4 ...
Score for fold 4: loss of 5.066501617431641; mae of 5.066501617431641
------------------------------------------------------------------------
Training for fold 5 ...
Score for fold 5: loss of 5.848586559295654; mae of 5.848586559295654
------------------------------------------------------------------------
Training for fold 6 ...
Score for fold 6: loss of 4.4897613525390625; mae of 4.489761352539062

In [26]:
# SOAP descriptors, Mordred descriptors, enthalpy of reaction
model = build_model_4(11506)
inputs = np.array(data_soap_DeltaH)
mae_per_fold_soap_DeltaH_4, loss_per_fold_soap_DeltaH_4 = K_fold_cross_validation(
    inputs, data_soap_DeltaH_target, 10, model, mixed_dtypes=False, loss='mae', no_epochs=30, batch_size=32
)

------------------------------------------------------------------------
Training for fold 1 ...
Score for fold 1: loss of 8.034205436706543; mae of 8.034205436706543
------------------------------------------------------------------------
Training for fold 2 ...
Score for fold 2: loss of 6.620030403137207; mae of 6.620030403137207
------------------------------------------------------------------------
Training for fold 3 ...
Score for fold 3: loss of 5.6546454429626465; mae of 5.6546454429626465
------------------------------------------------------------------------
Training for fold 4 ...
Score for fold 4: loss of 4.735958099365234; mae of 4.735958099365234
------------------------------------------------------------------------
Training for fold 5 ...
Score for fold 5: loss of 5.143008232116699; mae of 5.143008232116699
------------------------------------------------------------------------
Training for fold 6 ...
Score for fold 6: loss of 4.523268222808838; mae of 4.523268222808

In [27]:
# LMBTR descriptors, Mordred descriptors
model = build_model_4(8239)
inputs = np.array(data_lmbtr_se_noBDEs)
mae_per_fold_lmbtr_se_noBDEs_4, loss_per_fold_lmbtr_se_noBDEs_4 = K_fold_cross_validation(
    inputs, data_lmbtr_se_noBDEs_target, 10, model, mixed_dtypes=False, loss='mae', no_epochs=30, batch_size=32
)

------------------------------------------------------------------------
Training for fold 1 ...
Score for fold 1: loss of 8.576237678527832; mae of 8.576237678527832
------------------------------------------------------------------------
Training for fold 2 ...
Score for fold 2: loss of 5.364546775817871; mae of 5.364546775817871
------------------------------------------------------------------------
Training for fold 3 ...
Score for fold 3: loss of 5.0186333656311035; mae of 5.0186333656311035
------------------------------------------------------------------------
Training for fold 4 ...
Score for fold 4: loss of 4.217813491821289; mae of 4.217813491821289
------------------------------------------------------------------------
Training for fold 5 ...
Score for fold 5: loss of 4.273894786834717; mae of 4.273894786834717
------------------------------------------------------------------------
Training for fold 6 ...
Score for fold 6: loss of 4.006100654602051; mae of 4.006100654602

In [28]:
# LMBTR descriptors, Mordred descriptors, enthalpy of reaction
model = build_model_4(8240)
inputs = np.array(data_lmbtr_DeltaH)
mae_per_fold_lmbtr_DeltaH_4, loss_per_fold_lmbtr_DeltaH_4 = K_fold_cross_validation(
    inputs, data_lmbtr_DeltaH_target, 10, model, mixed_dtypes=False, loss='mae', no_epochs=30, batch_size=32
)

------------------------------------------------------------------------
Training for fold 1 ...
Score for fold 1: loss of 8.787492752075195; mae of 8.787492752075195
------------------------------------------------------------------------
Training for fold 2 ...
Score for fold 2: loss of 5.272426605224609; mae of 5.272426605224609
------------------------------------------------------------------------
Training for fold 3 ...
Score for fold 3: loss of 4.747499942779541; mae of 4.747499942779541
------------------------------------------------------------------------
Training for fold 4 ...
Score for fold 4: loss of 4.104258060455322; mae of 4.104258060455322
------------------------------------------------------------------------
Training for fold 5 ...
Score for fold 5: loss of 4.679937362670898; mae of 4.679937362670898
------------------------------------------------------------------------
Training for fold 6 ...
Score for fold 6: loss of 3.914050579071045; mae of 3.91405105590820

In [29]:
df_hyper_opt_more_layers = pd.read_csv('cross_val_scores_hyper_opt_more_layers.csv', index_col=0)

new_scores = pd.DataFrame({'Model': [
    *['Model 1 (6 layers)' for i in range(len(mae_per_fold_soap_se_nomordred_4))],
    *['Model 2 (6 layers)' for i in range(len(mae_per_fold_soap_se_noBDEs_4))],
    *['Model 3 (6 layers)' for i in range(len(mae_per_fold_soap_DeltaH_4))],
    *['Model 4 (6 layers)' for i in range(len(mae_per_fold_lmbtr_se_noBDEs_4))],
    *['Model 5 (6 layers)' for i in range(len(mae_per_fold_lmbtr_DeltaH_4))]
], 'MAE [kcal/mol]': [
    *mae_per_fold_soap_se_nomordred_4, *mae_per_fold_soap_se_noBDEs_4, *mae_per_fold_soap_DeltaH_4,
    *mae_per_fold_lmbtr_se_noBDEs_4, *mae_per_fold_lmbtr_DeltaH_4
]})
df_hyper_opt_more_layers = pd.concat([df_hyper_opt_more_layers, new_scores])
df_hyper_opt_more_layers.to_csv('cross_val_scores_hyper_opt_more_layers.csv')

## 7 layers

In [30]:
# SOAP descriptors, Mordred descriptors
model = build_model_5(11491)
inputs = np.array(data_soap_se_nomordred)
mae_per_fold_soap_se_nomordred_5, loss_per_fold_soap_se_nomordred_5 = K_fold_cross_validation(
    inputs, data_soap_se_nomordred_target, 10, model, mixed_dtypes=False, loss='mae', no_epochs=30, batch_size=32
)

------------------------------------------------------------------------
Training for fold 1 ...
Score for fold 1: loss of 8.335159301757812; mae of 8.335159301757812
------------------------------------------------------------------------
Training for fold 2 ...
Score for fold 2: loss of 6.26959228515625; mae of 6.26959228515625
------------------------------------------------------------------------
Training for fold 3 ...
Score for fold 3: loss of 5.6732683181762695; mae of 5.6732683181762695
------------------------------------------------------------------------
Training for fold 4 ...
Score for fold 4: loss of 5.1814069747924805; mae of 5.1814069747924805
------------------------------------------------------------------------
Training for fold 5 ...
Score for fold 5: loss of 5.159209728240967; mae of 5.159209728240967
------------------------------------------------------------------------
Training for fold 6 ...
Score for fold 6: loss of 4.371696472167969; mae of 4.371696472167

In [31]:
# SOAP descriptors, BDEs
model = build_model_5(11505)
inputs = np.array(data_soap_se_noBDEs)
mae_per_fold_soap_se_noBDEs_5, loss_per_fold_soap_se_noBDEs_5 = K_fold_cross_validation(
    inputs, data_soap_se_noBDEs_target, 10, model, mixed_dtypes=False, loss='mae', no_epochs=30, batch_size=32
)

------------------------------------------------------------------------
Training for fold 1 ...
Score for fold 1: loss of 8.268417358398438; mae of 8.268417358398438
------------------------------------------------------------------------
Training for fold 2 ...
Score for fold 2: loss of 6.487494468688965; mae of 6.487494468688965
------------------------------------------------------------------------
Training for fold 3 ...
Score for fold 3: loss of 5.577263355255127; mae of 5.577263355255127
------------------------------------------------------------------------
Training for fold 4 ...
Score for fold 4: loss of 4.843913555145264; mae of 4.843913555145264
------------------------------------------------------------------------
Training for fold 5 ...
Score for fold 5: loss of 5.77504825592041; mae of 5.77504825592041
------------------------------------------------------------------------
Training for fold 6 ...
Score for fold 6: loss of 4.6505537033081055; mae of 4.650553703308105

In [32]:
# SOAP descriptors, Mordred descriptors, enthalpy of reaction
model = build_model_5(11506)
inputs = np.array(data_soap_DeltaH)
mae_per_fold_soap_DeltaH_5, loss_per_fold_soap_DeltaH_5 = K_fold_cross_validation(
    inputs, data_soap_DeltaH_target, 10, model, mixed_dtypes=False, loss='mae', no_epochs=30, batch_size=32
)

------------------------------------------------------------------------
Training for fold 1 ...
Score for fold 1: loss of 8.583242416381836; mae of 8.583242416381836
------------------------------------------------------------------------
Training for fold 2 ...
Score for fold 2: loss of 6.170719146728516; mae of 6.170719146728516
------------------------------------------------------------------------
Training for fold 3 ...
Score for fold 3: loss of 5.619753360748291; mae of 5.619753360748291
------------------------------------------------------------------------
Training for fold 4 ...
Score for fold 4: loss of 4.66050910949707; mae of 4.66050910949707
------------------------------------------------------------------------
Training for fold 5 ...
Score for fold 5: loss of 5.299604415893555; mae of 5.299604415893555
------------------------------------------------------------------------
Training for fold 6 ...
Score for fold 6: loss of 4.5752997398376465; mae of 4.575299739837646

In [33]:
# LMBTR descriptors, Mordred descriptors
model = build_model_5(8239)
inputs = np.array(data_lmbtr_se_noBDEs)
mae_per_fold_lmbtr_se_noBDEs_5, loss_per_fold_lmbtr_se_noBDEs_5 = K_fold_cross_validation(
    inputs, data_lmbtr_se_noBDEs_target, 10, model, mixed_dtypes=False, loss='mae', no_epochs=30, batch_size=32
)

------------------------------------------------------------------------
Training for fold 1 ...
Score for fold 1: loss of 8.618277549743652; mae of 8.618277549743652
------------------------------------------------------------------------
Training for fold 2 ...
Score for fold 2: loss of 5.51271915435791; mae of 5.51271915435791
------------------------------------------------------------------------
Training for fold 3 ...
Score for fold 3: loss of 4.869907855987549; mae of 4.869907855987549
------------------------------------------------------------------------
Training for fold 4 ...
Score for fold 4: loss of 4.265510082244873; mae of 4.265510082244873
------------------------------------------------------------------------
Training for fold 5 ...
Score for fold 5: loss of 4.483770370483398; mae of 4.483770370483398
------------------------------------------------------------------------
Training for fold 6 ...
Score for fold 6: loss of 4.739161968231201; mae of 4.739161968231201


In [34]:
# LMBTR descriptors, Mordred descriptors, enthalpy of reaction
model = build_model_5(8240)
inputs = np.array(data_lmbtr_DeltaH)
mae_per_fold_lmbtr_DeltaH_5, loss_per_fold_lmbtr_DeltaH_5 = K_fold_cross_validation(
    inputs, data_lmbtr_DeltaH_target, 10, model, mixed_dtypes=False, loss='mae', no_epochs=30, batch_size=32
)

------------------------------------------------------------------------
Training for fold 1 ...
Score for fold 1: loss of 8.943923950195312; mae of 8.943923950195312
------------------------------------------------------------------------
Training for fold 2 ...
Score for fold 2: loss of 5.341494560241699; mae of 5.341494560241699
------------------------------------------------------------------------
Training for fold 3 ...
Score for fold 3: loss of 4.954117774963379; mae of 4.954117774963379
------------------------------------------------------------------------
Training for fold 4 ...
Score for fold 4: loss of 4.273805618286133; mae of 4.273805618286133
------------------------------------------------------------------------
Training for fold 5 ...
Score for fold 5: loss of 4.9594502449035645; mae of 4.9594502449035645
------------------------------------------------------------------------
Training for fold 6 ...
Score for fold 6: loss of 4.296631813049316; mae of 4.296631813049

In [35]:
df_hyper_opt_more_layers = pd.read_csv('cross_val_scores_hyper_opt_more_layers.csv', index_col=0)

new_scores = pd.DataFrame({'Model': [
    *['Model 1 (7 layers)' for i in range(len(mae_per_fold_soap_se_nomordred_5))],
    *['Model 2 (7 layers)' for i in range(len(mae_per_fold_soap_se_noBDEs_5))],
    *['Model 3 (7 layers)' for i in range(len(mae_per_fold_soap_DeltaH_5))],
    *['Model 4 (7 layers)' for i in range(len(mae_per_fold_lmbtr_se_noBDEs_5))],
    *['Model 5 (7 layers)' for i in range(len(mae_per_fold_lmbtr_DeltaH_5))]
], 'MAE [kcal/mol]': [
    *mae_per_fold_soap_se_nomordred_5, *mae_per_fold_soap_se_noBDEs_5, *mae_per_fold_soap_DeltaH_5,
    *mae_per_fold_lmbtr_se_noBDEs_5, *mae_per_fold_lmbtr_DeltaH_5
]})
df_hyper_opt_more_layers = pd.concat([df_hyper_opt_more_layers, new_scores])
df_hyper_opt_more_layers.to_csv('cross_val_scores_hyper_opt_more_layers.csv')

## 8 layers

In [36]:
# SOAP descriptors, Mordred descriptors
model = build_model_6(11491)
inputs = np.array(data_soap_se_nomordred)
mae_per_fold_soap_se_nomordred_6, loss_per_fold_soap_se_nomordred_6 = K_fold_cross_validation(
    inputs, data_soap_se_nomordred_target, 10, model, mixed_dtypes=False, loss='mae', no_epochs=30, batch_size=32
)

------------------------------------------------------------------------
Training for fold 1 ...
Score for fold 1: loss of 8.557238578796387; mae of 8.557238578796387
------------------------------------------------------------------------
Training for fold 2 ...
Score for fold 2: loss of 6.727097511291504; mae of 6.727097511291504
------------------------------------------------------------------------
Training for fold 3 ...
Score for fold 3: loss of 5.869279384613037; mae of 5.869279384613037
------------------------------------------------------------------------
Training for fold 4 ...
Score for fold 4: loss of 4.847896575927734; mae of 4.847896575927734
------------------------------------------------------------------------
Training for fold 5 ...
Score for fold 5: loss of 5.704553127288818; mae of 5.704553127288818
------------------------------------------------------------------------
Training for fold 6 ...
Score for fold 6: loss of 4.7884321212768555; mae of 4.7884321212768

In [37]:
# SOAP descriptors, BDEs
model = build_model_6(11505)
inputs = np.array(data_soap_se_noBDEs)
mae_per_fold_soap_se_noBDEs_6, loss_per_fold_soap_se_noBDEs_6 = K_fold_cross_validation(
    inputs, data_soap_se_noBDEs_target, 10, model, mixed_dtypes=False, loss='mae', no_epochs=30, batch_size=32
)

------------------------------------------------------------------------
Training for fold 1 ...
Score for fold 1: loss of 8.146730422973633; mae of 8.146730422973633
------------------------------------------------------------------------
Training for fold 2 ...
Score for fold 2: loss of 6.420041084289551; mae of 6.420041084289551
------------------------------------------------------------------------
Training for fold 3 ...
Score for fold 3: loss of 5.943288326263428; mae of 5.943288326263428
------------------------------------------------------------------------
Training for fold 4 ...
Score for fold 4: loss of 4.93197774887085; mae of 4.93197774887085
------------------------------------------------------------------------
Training for fold 5 ...
Score for fold 5: loss of 5.789530277252197; mae of 5.789530277252197
------------------------------------------------------------------------
Training for fold 6 ...
Score for fold 6: loss of 4.934604644775391; mae of 4.934604644775391


In [38]:
# SOAP descriptors, Mordred descriptors, enthalpy of reaction
model = build_model_6(11506)
inputs = np.array(data_soap_DeltaH)
mae_per_fold_soap_DeltaH_6, loss_per_fold_soap_DeltaH_6 = K_fold_cross_validation(
    inputs, data_soap_DeltaH_target, 10, model, mixed_dtypes=False, loss='mae', no_epochs=30, batch_size=32
)

------------------------------------------------------------------------
Training for fold 1 ...
Score for fold 1: loss of 8.50295639038086; mae of 8.50295639038086
------------------------------------------------------------------------
Training for fold 2 ...
Score for fold 2: loss of 6.716197490692139; mae of 6.716197490692139
------------------------------------------------------------------------
Training for fold 3 ...
Score for fold 3: loss of 5.800463676452637; mae of 5.800463676452637
------------------------------------------------------------------------
Training for fold 4 ...
Score for fold 4: loss of 4.739253044128418; mae of 4.739253044128418
------------------------------------------------------------------------
Training for fold 5 ...
Score for fold 5: loss of 5.41719388961792; mae of 5.41719388961792
------------------------------------------------------------------------
Training for fold 6 ...
Score for fold 6: loss of 4.413856506347656; mae of 4.413856506347656
--

In [39]:
# LMBTR descriptors, Mordred descriptors
model = build_model_6(8239)
inputs = np.array(data_lmbtr_se_noBDEs)
mae_per_fold_lmbtr_se_noBDEs_6, loss_per_fold_lmbtr_se_noBDEs_6 = K_fold_cross_validation(
    inputs, data_lmbtr_se_noBDEs_target, 10, model, mixed_dtypes=False, loss='mae', no_epochs=30, batch_size=32
)

------------------------------------------------------------------------
Training for fold 1 ...
Score for fold 1: loss of 8.803272247314453; mae of 8.803272247314453
------------------------------------------------------------------------
Training for fold 2 ...
Score for fold 2: loss of 6.094285011291504; mae of 6.094285011291504
------------------------------------------------------------------------
Training for fold 3 ...
Score for fold 3: loss of 5.468090057373047; mae of 5.468090057373047
------------------------------------------------------------------------
Training for fold 4 ...
Score for fold 4: loss of 4.543014049530029; mae of 4.543014049530029
------------------------------------------------------------------------
Training for fold 5 ...
Score for fold 5: loss of 4.4280829429626465; mae of 4.4280829429626465
------------------------------------------------------------------------
Training for fold 6 ...
Score for fold 6: loss of 4.565420627593994; mae of 4.565420627593

In [40]:
# LMBTR descriptors, Mordred descriptors, enthalpy of reaction
model = build_model_6(8240)
inputs = np.array(data_lmbtr_DeltaH)
mae_per_fold_lmbtr_DeltaH_6, loss_per_fold_lmbtr_DeltaH_6 = K_fold_cross_validation(
    inputs, data_lmbtr_DeltaH_target, 10, model, mixed_dtypes=False, loss='mae', no_epochs=30, batch_size=32
)

------------------------------------------------------------------------
Training for fold 1 ...
Score for fold 1: loss of 8.428010940551758; mae of 8.428010940551758
------------------------------------------------------------------------
Training for fold 2 ...
Score for fold 2: loss of 5.674784183502197; mae of 5.674784183502197
------------------------------------------------------------------------
Training for fold 3 ...
Score for fold 3: loss of 4.787662982940674; mae of 4.787662982940674
------------------------------------------------------------------------
Training for fold 4 ...
Score for fold 4: loss of 4.421676158905029; mae of 4.421676158905029
------------------------------------------------------------------------
Training for fold 5 ...
Score for fold 5: loss of 4.165317535400391; mae of 4.165317535400391
------------------------------------------------------------------------
Training for fold 6 ...
Score for fold 6: loss of 4.093502521514893; mae of 4.09350252151489

In [41]:
df_hyper_opt_more_layers = pd.read_csv('cross_val_scores_hyper_opt_more_layers.csv', index_col=0)

new_scores = pd.DataFrame({'Model': [
    *['Model 1 (8 layers)' for i in range(len(mae_per_fold_soap_se_nomordred_6))],
    *['Model 2 (8 layers)' for i in range(len(mae_per_fold_soap_se_noBDEs_6))],
    *['Model 3 (8 layers)' for i in range(len(mae_per_fold_soap_DeltaH_6))],
    *['Model 4 (8 layers)' for i in range(len(mae_per_fold_lmbtr_se_noBDEs_6))],
    *['Model 5 (8 layers)' for i in range(len(mae_per_fold_lmbtr_DeltaH_6))]
], 'MAE [kcal/mol]': [
    *mae_per_fold_soap_se_nomordred_6, *mae_per_fold_soap_se_noBDEs_6, *mae_per_fold_soap_DeltaH_6,
    *mae_per_fold_lmbtr_se_noBDEs_6, *mae_per_fold_lmbtr_DeltaH_6
]})
df_hyper_opt_more_layers = pd.concat([df_hyper_opt_more_layers, new_scores])
df_hyper_opt_more_layers.to_csv('cross_val_scores_hyper_opt_more_layers.csv')

# Find the optimal number of units in the layers

In [10]:
# hypermodel with 4 layers
def model_builder_4layers(hp):
  model = tf.keras.Sequential()

  # Tune the number of units
  hp_units_1 = hp.Int('units_1', min_value=32, max_value=1024, step=32)
  hp_units_2 = hp.Int('units_2', min_value=32, max_value=512, step=32)
  hp_units_3 = hp.Int('units_3', min_value=32, max_value=256, step=32)
  model.add(tf.keras.layers.Dense(units=hp_units_1, activation='relu'))
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.Dense(units=hp_units_2, activation='relu'))
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.Dense(units=hp_units_3, activation='relu'))
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.Dense(1))

  lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
    0.001, decay_steps=inputs.shape[0]*0.8/32*1000, decay_rate=1, staircase=False
  )
  optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
  model.compile(optimizer=optimizer, loss='mae', metrics=['mae'])

  return model

# hypermodel with 5 layers
def model_builder_5layers(hp):
  model = tf.keras.Sequential()

  # Tune the number of units
  hp_units_1 = hp.Int('units_1', min_value=32, max_value=1024, step=32)
  hp_units_2 = hp.Int('units_2', min_value=32, max_value=512, step=32)
  hp_units_3 = hp.Int('units_3', min_value=32, max_value=256, step=32)
  hp_units_4 = hp.Int('units_4', min_value=32, max_value=128, step=32)
  model.add(tf.keras.layers.Dense(units=hp_units_1, activation='relu'))
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.Dense(units=hp_units_2, activation='relu'))
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.Dense(units=hp_units_3, activation='relu'))
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.Dense(units=hp_units_4, activation='relu'))
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.Dense(1))

  lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
    0.001, decay_steps=inputs.shape[0]*0.8/32*1000, decay_rate=1, staircase=False
  )
  optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
  model.compile(optimizer=optimizer, loss='mae', metrics=['mae'])

  return model

# class to perform 10 fold crossvalidated hyperparameter optimization
class CVTuner(kt.engine.tuner.Tuner):

  def run_trial(self, trial, inputs, targets, batch_size=32, epochs=30):
    kfold = KFold(n_splits=10, shuffle=True, random_state = 1)
    callbacks=tf.keras.callbacks.EarlyStopping(monitor='mae', patience=20)
    val_losses = []

    for train, test in kfold.split(inputs, targets):
      model = self.hypermodel.build(trial.hyperparameters)
      model.fit(inputs[train], targets[train], batch_size=batch_size, epochs=epochs, callbacks=callbacks, verbose=0)
      val_losses.append(model.evaluate(inputs[test], targets[test]))

    self.oracle.update_trial(trial.trial_id, {'val_loss': np.mean(val_losses)})

In [10]:
# SOAP descriptors, Mordred descriptors
inputs = np.array(data_soap_se_nomordred)

tuner = CVTuner(
  hypermodel=model_builder_5layers,
  oracle=kt.oracles.HyperbandOracle(
    objective='val_loss',
    hyperband_iterations=2,
    seed=1,
    hyperparameters=None
  ),
  directory='hyper_opt_log',
  project_name='soap_se_nomordred'
)

tuner.search(inputs, data_soap_se_nomordred_target)

best_hps=tuner.get_best_hyperparameters()[0]
print('Optimal number of units are {}, {}, {}.'.format(best_hps.get('units_1'), best_hps.get('units_2'), best_hps.get('units_3')))


Trial 7 Complete [00h 02m 12s]
val_loss: 8.214355754852296

Best val_loss So Far: 8.009508085250854
Total elapsed time: 00h 14m 17s

Search: Running Trial #8

Hyperparameter    |Value             |Best Value So Far 
units_1           |416               |160               
units_2           |768               |992               
units_3           |672               |256               
units_4           |160               |256               
tuner/epochs      |2                 |2                 
tuner/initial_e...|0                 |0                 
tuner/bracket     |4                 |4                 
tuner/round       |0                 |0                 



Epoch 1/30
173/173 [==============================] - 1s 3ms/step - loss: 28.4271 - mae: 28.4271
Epoch 2/30
173/173 [==============================] - 0s 3ms/step - loss: 13.3074 - mae: 13.3074
Epoch 3/30
173/173 [==============================] - 0s 3ms/step - loss: 11.4809 - mae: 11.4809
Epoch 4/30
173/173 [==============================] - 0s 3ms/step - loss: 10.0284 - mae: 10.0284
Epoch 5/30
173/173 [==============================] - 0s 3ms/step - loss: 9.2632 - mae: 9.2632
Epoch 6/30
173/173 [==============================] - 0s 3ms/step - loss: 9.0560 - mae: 9.0560
Epoch 7/30
173/173 [==============================] - 0s 3ms/step - loss: 8.5802 - mae: 8.5802
Epoch 8/30
173/173 [==============================] - 0s 3ms/step - loss: 7.8591 - mae: 7.8591
Epoch 9/30
173/173 [==============================] - 0s 3ms/step - loss: 7.2601 - mae: 7.2601
Epoch 10/30
173/173 [==============================] - 0s 3ms/step - loss: 7.4549 - mae: 7.4549
Epoch 11/30
173/173 [====================

A corresponding script was run to find the optimal number of units for each layer.

In [8]:
# evaluate optimized models
def build_model4_opt_number(input_shape, no_nodes):
    input = tf.keras.layers.Input(shape=(input_shape), dtype='float32')
    out = tf.keras.layers.Dense(no_nodes[0], activation='relu')(input)
    out = tf.keras.layers.BatchNormalization()(out)
    out = tf.keras.layers.Dense(no_nodes[1], activation='relu')(out)
    out = tf.keras.layers.BatchNormalization()(out)
    out = tf.keras.layers.Dense(no_nodes[2], activation='relu')(out)
    out = tf.keras.layers.BatchNormalization()(out)
    out = tf.keras.layers.Dense(1)(out)
    model = tf.keras.Model(input, out)
    return model

def build_model5_opt_number(input_shape, no_nodes):
    input = tf.keras.layers.Input(shape=(input_shape), dtype='float32')
    out = tf.keras.layers.Dense(no_nodes[0], activation='relu')(input)
    out = tf.keras.layers.BatchNormalization()(out)
    out = tf.keras.layers.Dense(no_nodes[1], activation='relu')(out)
    out = tf.keras.layers.BatchNormalization()(out)
    out = tf.keras.layers.Dense(no_nodes[2], activation='relu')(out)
    out = tf.keras.layers.BatchNormalization()(out)
    out = tf.keras.layers.Dense(no_nodes[3], activation='relu')(out)
    out = tf.keras.layers.BatchNormalization()(out)
    out = tf.keras.layers.Dense(1)(out)
    model = tf.keras.Model(input, out)
    return model

In [11]:
# SOAP descriptors, Mordred descriptors
model = build_model5_opt_number(11491, [352, 480, 224, 96])
inputs = np.array(data_soap_se_nomordred)
mae_per_fold_soap_se_nomordred_opt_number, loss_per_fold_soap_se_nomordred_opt_number = K_fold_cross_validation(
    inputs, data_soap_se_nomordred_target, 10, model, mixed_dtypes=False, loss='mae', no_epochs=30, batch_size=32
)

------------------------------------------------------------------------
Training for fold 1 ...


2022-02-13 22:20:08.325824: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-02-13 22:20:08.346096: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3192770000 Hz
2022-02-13 22:20:09.231881: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10


Score for fold 1: loss of 8.09354019165039; mae of 8.09354019165039
------------------------------------------------------------------------
Training for fold 2 ...
Score for fold 2: loss of 5.9194769859313965; mae of 5.9194769859313965
------------------------------------------------------------------------
Training for fold 3 ...
Score for fold 3: loss of 5.515296936035156; mae of 5.515296936035156
------------------------------------------------------------------------
Training for fold 4 ...
Score for fold 4: loss of 4.445985794067383; mae of 4.445985794067383
------------------------------------------------------------------------
Training for fold 5 ...
Score for fold 5: loss of 4.862082004547119; mae of 4.862082004547119
------------------------------------------------------------------------
Training for fold 6 ...
Score for fold 6: loss of 4.2825751304626465; mae of 4.2825751304626465
------------------------------------------------------------------------
Training for fold 7 

In [12]:
np.save('mae_per_fold_soap_se_nomordred_opt_number', mae_per_fold_soap_se_nomordred_opt_number)

In [9]:
# SOAP descriptors, Mordred descriptors, no BDEs
model = build_model5_opt_number(11505, [480, 448, 96, 64])
inputs = np.array(data_soap_se_noBDEs)
mae_per_fold_soap_se_noBDEs_opt_number, loss_per_fold_soap_se_noBDEs_opt_number = K_fold_cross_validation(
    inputs, data_soap_se_noBDEs_target, 10, model, mixed_dtypes=False, loss='mae', no_epochs=30, batch_size=32
)

2022-02-15 03:34:37.048826: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-02-15 03:34:37.050103: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-02-15 03:34:37.079780: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_COMPAT_NOT_SUPPORTED_ON_DEVICE: forward compatibility was attempted on non supported HW
2022-02-15 03:34:37.079821: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: rh02572.villa-bosch.de
2022-02-15 03:34:37.079832: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: rh02572.villa-bosch.de
2022-02-15 03:34:37.079945: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 470.103.1
2022-02-15 03:34:37.079984: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is:

------------------------------------------------------------------------
Training for fold 1 ...


2022-02-15 03:34:38.090975: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-02-15 03:34:38.122119: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3192770000 Hz


Score for fold 1: loss of 8.033854484558105; mae of 8.033854484558105
------------------------------------------------------------------------
Training for fold 2 ...
Score for fold 2: loss of 5.973505973815918; mae of 5.973505973815918
------------------------------------------------------------------------
Training for fold 3 ...
Score for fold 3: loss of 5.384171009063721; mae of 5.384171009063721
------------------------------------------------------------------------
Training for fold 4 ...
Score for fold 4: loss of 4.39673376083374; mae of 4.39673376083374
------------------------------------------------------------------------
Training for fold 5 ...
Score for fold 5: loss of 4.852954864501953; mae of 4.852954864501953
------------------------------------------------------------------------
Training for fold 6 ...
Score for fold 6: loss of 4.8246893882751465; mae of 4.8246893882751465
------------------------------------------------------------------------
Training for fold 7 ..

In [10]:
np.save('mae_per_fold_soap_se_noBDEs_opt_number', mae_per_fold_soap_se_noBDEs_opt_number)

In [9]:
# SOAP descriptors, Mordred descriptors, enthalpy of reaction
model = build_model4_opt_number(11506, [704, 448, 128])
inputs = np.array(data_soap_DeltaH)

mae_per_fold_soap_DeltaH_opt_number, loss_per_fold_soap_DeltaH_opt_number = K_fold_cross_validation(
    inputs, data_soap_DeltaH_target, 10, model, mixed_dtypes=False, loss='mae', no_epochs=30, batch_size=32
)

2022-02-26 20:11:46.928837: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-02-26 20:11:46.931124: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-02-26 20:11:46.972856: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-26 20:11:46.973267: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 2060 computeCapability: 7.5
coreClock: 1.68GHz coreCount: 30 deviceMemorySize: 5.79GiB deviceMemoryBandwidth: 312.97GiB/s
2022-02-26 20:11:46.973299: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-02-26 20:11:47.328043: I tensorflow/stream_executor/platform/def

------------------------------------------------------------------------
Training for fold 1 ...


2022-02-26 20:11:54.388018: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 203794272 exceeds 10% of free system memory.
2022-02-26 20:11:54.525693: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-02-26 20:11:54.543476: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3192740000 Hz
2022-02-26 20:11:55.297051: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2022-02-26 20:11:58.804406: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 50948568 exceeds 10% of free system memory.
2022-02-26 20:12:14.354983: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 28304760 exceeds 10% of free system memory.


Score for fold 1: loss of 7.934071063995361; mae of 7.934071063995361
------------------------------------------------------------------------
Training for fold 2 ...


2022-02-26 20:12:15.488158: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 203794272 exceeds 10% of free system memory.
2022-02-26 20:12:16.640709: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 50948568 exceeds 10% of free system memory.


Score for fold 2: loss of 5.973849773406982; mae of 5.973849773406982
------------------------------------------------------------------------
Training for fold 3 ...
Score for fold 3: loss of 5.430459976196289; mae of 5.430459976196289
------------------------------------------------------------------------
Training for fold 4 ...
Score for fold 4: loss of 4.617372989654541; mae of 4.617372989654541
------------------------------------------------------------------------
Training for fold 5 ...
Score for fold 5: loss of 4.771920680999756; mae of 4.771920680999756
------------------------------------------------------------------------
Training for fold 6 ...
Score for fold 6: loss of 4.298646450042725; mae of 4.298646450042725
------------------------------------------------------------------------
Training for fold 7 ...
Score for fold 7: loss of 3.917611837387085; mae of 3.917611837387085
------------------------------------------------------------------------
Training for fold 8 ..

In [10]:
np.save('mae_per_fold_soap_DeltaH_opt_number', mae_per_fold_soap_DeltaH_opt_number)

In [11]:
# LMBTR descriptors, Mordred descriptors
inputs = np.array(data_lmbtr_se_noBDEs)
model = build_model5_opt_number(8239, [736, 192, 96, 96])

mae_per_fold_lmbtr_se_noBDEs_opt_number, loss_per_fold_lmbtr_se_noBDEs_opt_number = K_fold_cross_validation(
    inputs, data_lmbtr_se_noBDEs_target, 10, model, mixed_dtypes=False, loss='mae', no_epochs=30, batch_size=32
)

------------------------------------------------------------------------
Training for fold 1 ...
Score for fold 1: loss of 8.558971405029297; mae of 8.558971405029297
------------------------------------------------------------------------
Training for fold 2 ...
Score for fold 2: loss of 5.687483310699463; mae of 5.687483310699463
------------------------------------------------------------------------
Training for fold 3 ...
Score for fold 3: loss of 4.775956630706787; mae of 4.775956630706787
------------------------------------------------------------------------
Training for fold 4 ...
Score for fold 4: loss of 4.244556903839111; mae of 4.244556903839111
------------------------------------------------------------------------
Training for fold 5 ...
Score for fold 5: loss of 4.668138027191162; mae of 4.668138027191162
------------------------------------------------------------------------
Training for fold 6 ...
Score for fold 6: loss of 4.669400691986084; mae of 4.66940069198608

In [12]:
np.save('mae_per_fold_lmbtr_se_noBDEs_opt_number', mae_per_fold_lmbtr_se_noBDEs_opt_number)

In [13]:
# LMBTR descriptors, Mordred descriptors, enthalpy of reaction
inputs = np.array(data_lmbtr_DeltaH)
model = build_model4_opt_number(8240, [352, 128, 128])

mae_per_fold_lmbtr_DeltaH_opt_number, loss_per_fold_lmbtr_DeltaH_opt_number = K_fold_cross_validation(
    inputs, data_lmbtr_DeltaH_target, 10, model, mixed_dtypes=False, loss='mae', no_epochs=30, batch_size=32
)

------------------------------------------------------------------------
Training for fold 1 ...
Score for fold 1: loss of 8.211527824401855; mae of 8.211527824401855
------------------------------------------------------------------------
Training for fold 2 ...
Score for fold 2: loss of 5.153715133666992; mae of 5.153715133666992
------------------------------------------------------------------------
Training for fold 3 ...
Score for fold 3: loss of 4.408202171325684; mae of 4.408202171325684
------------------------------------------------------------------------
Training for fold 4 ...
Score for fold 4: loss of 3.7796719074249268; mae of 3.7796719074249268
------------------------------------------------------------------------
Training for fold 5 ...
Score for fold 5: loss of 4.317324161529541; mae of 4.317324161529541
------------------------------------------------------------------------
Training for fold 6 ...
Score for fold 6: loss of 3.9629247188568115; mae of 3.96292471885

In [14]:
np.save('mae_per_fold_lmbtr_DeltaH_opt_number', mae_per_fold_lmbtr_DeltaH_opt_number)

# Implement skip connections

In [ ]:
def build_4layer_model_with_skipcon(input_shape, no_nodes):
    input = tf.keras.layers.Input(shape=(input_shape), dtype='float32')
    a = tf.keras.layers.Dense(no_nodes[0], activation='relu')(input)
    
    b = tf.keras.layers.BatchNormalization()(a)
    b = tf.keras.layers.Dense(no_nodes[1], activation='relu')(b)

    c = tf.keras.layers.Concatenate()([a, b])
    c = tf.keras.layers.BatchNormalization()(c)
    c = tf.keras.layers.Dense(no_nodes[2], activation='relu')(c)

    d = tf.keras.layers.Concatenate()([a, b, c])
    d = tf.keras.layers.BatchNormalization()(d)
    out = tf.keras.layers.Dense(1)(d)
    model = tf.keras.Model(input, out)
    return model

def build_5layer_model_with_skipcon(input_shape, no_nodes):
    input = tf.keras.layers.Input(shape=(input_shape), dtype='float32')
    a = tf.keras.layers.Dense(no_nodes[0], activation='relu')(input)
    
    b = tf.keras.layers.BatchNormalization()(a)
    b = tf.keras.layers.Dense(no_nodes[1], activation='relu')(b)

    c = tf.keras.layers.Concatenate()([a, b])
    c = tf.keras.layers.BatchNormalization()(c)
    c = tf.keras.layers.Dense(no_nodes[2], activation='relu')(c)
    
    d = tf.keras.layers.Concatenate()([a, b, c])
    d = tf.keras.layers.BatchNormalization()(d)
    d = tf.keras.layers.Dense(no_nodes[3], activation='relu')(d)

    e = tf.keras.layers.Concatenate()([a, b, c, d])
    e = tf.keras.layers.BatchNormalization()(e)
    out = tf.keras.layers.Dense(1)(e)
    model = tf.keras.Model(input, out)
    return model

In [1]:
from graphviz import Digraph

In [28]:
dot = Digraph(
    engine='neato',
    node_attr={'shape': 'box', 'fontname': 'Roboto Light'},
    graph_attr={'fontname': 'Roboto Light', 'splines': 'ortho'}
)

dot.attr('node', shape='box3d')
dot.node('A', 'Input Layer', pos='0,8!')
dot.attr('node', shape='box')
dot.node('B', 'Hidden Layer', pos='0,7!')
dot.node('C', 'Hidden Layer', pos='0,6!')
dot.node('D', 'Hidden Layer', pos='0,4!')
dot.node('E', 'Hidden Layer', pos='0,2!')
dot.attr('node', shape='box3d')
dot.node('F', 'Output Layer', pos='0,0!')
dot.attr('node', shape='box')

#Concatenation nodes
dot.attr('node', shape='circle')
dot.node('C1', '||', pos='0,5!')
dot.node('C2', '||', pos='0,3!')
dot.node('C3', '||', pos='0,1!')
dot.attr('node', shape='box')

dot.edge('A', 'B')
dot.edge('B', 'C')
dot.edge('B', 'C1')
dot.edge('C', 'C1')
dot.edge('C1', 'D')
dot.edge('B', 'C2')
dot.edge('C', 'C2')
dot.edge('D', 'C2')
dot.edge('C2', 'E')
dot.edge('B', 'C3')
dot.edge('C', 'C3')
dot.edge('D', 'C3')
dot.edge('E', 'C3')
dot.edge('C3', 'F')

dot.format = 'png'
dot.render('Skip_NN')

'Skip_NN.png'

# Implement attention

In [ ]:
from tensorflow.keras import backend as K

In [ ]:
#self-attention
class attention(tf.keras.layers.Layer):
    def __init__(self, out_shape=32):
        super(attention, self).__init__()
        self.out_shape = out_shape

    def build(self, input_shape):
        self.batch_size = input_shape[0]
        #key weights
        self.K = self.add_weight(name='key_weight', shape=(input_shape[1], 128), 
                               initializer='glorot_uniform', trainable=True)
        #key bias
        self.K_b = self.add_weight(name='key_bias', shape=(self.batch_size, 128), 
                               initializer='zeros', trainable=True)
        #query weights
        self.Q = self.add_weight(name='query_weight', shape=(input_shape[1], 128), 
                               initializer='glorot_uniform', trainable=True)
        #query bias
        self.Q_b = self.add_weight(name='query_bias', shape=(self.batch_size, 128), 
                               initializer='zeros', trainable=True)
        #value weights
        self.V = self.add_weight(name='value_weight', shape=(input_shape[1], self.out_shape), 
                               initializer='glorot_uniform', trainable=True)
        #value bias
        self.V_b = self.add_weight(name='value_bias', shape=(self.batch_size, self.out_shape), 
                               initializer='zeros', trainable=True)

    def call(self,x):
        keys = tf.matmul(x, self.K) + self.K_b
        queries =  tf.matmul(x, self.Q) + self.Q_b
        values = tf.matmul(x, self.V) + self.V_b
        #attention scores
        attn_scores = K.softmax(tf.matmul(queries, keys, transpose_b=True))
        weighted_values = tf.matmul(attn_scores, values)
        return weighted_values

#multi-head self-attention
class multihead_attention(tf.keras.layers.Layer):
    def __init__(self, out_shape=32):
        super(multihead_attention, self).__init__()
        self.out_shape = out_shape

    def build(self, input_shape):
        self.batch_size = input_shape[0]
        #key weights
        self.K = self.add_weight(name='key_weight', shape=(input_shape[1], 64), 
                               initializer='glorot_uniform', trainable=True)
        self.K1 = self.add_weight(name='key_weight1', shape=(64, 64), 
                               initializer='glorot_uniform', trainable=True)
        self.K2 = self.add_weight(name='key_weight2', shape=(64, 64), 
                               initializer='glorot_uniform', trainable=True)
        self.K3 = self.add_weight(name='key_weight3', shape=(64, 64), 
                               initializer='glorot_uniform', trainable=True)
        self.K4 = self.add_weight(name='key_weight4', shape=(64, 64), 
                               initializer='glorot_uniform', trainable=True)
        #key bias
        # self.K_b = self.add_weight(name='key_bias', shape=(self.batch_size, 128), 
        #                        initializer='zeros', trainable=True)
        #query weights
        self.Q = self.add_weight(name='query_weight', shape=(input_shape[1], 64), 
                               initializer='glorot_uniform', trainable=True)
        self.Q1 = self.add_weight(name='query_weight1', shape=(64, 64), 
                               initializer='glorot_uniform', trainable=True)
        self.Q2 = self.add_weight(name='query_weight2', shape=(64, 64), 
                               initializer='glorot_uniform', trainable=True)
        self.Q3 = self.add_weight(name='query_weight3', shape=(64, 64), 
                               initializer='glorot_uniform', trainable=True)
        self.Q4 = self.add_weight(name='query_weight4', shape=(64, 64), 
                               initializer='glorot_uniform', trainable=True)
        #query bias
        # self.Q_b = self.add_weight(name='query_bias', shape=(self.batch_size, 128), 
        #                        initializer='zeros', trainable=True)
        #value weights
        self.V = self.add_weight(name='value_weight1', shape=(input_shape[1], self.out_shape), 
                               initializer='glorot_uniform', trainable=True)
        self.V1 = self.add_weight(name='value_weight1', shape=(self.out_shape, self.out_shape), 
                               initializer='glorot_uniform', trainable=True)
        self.V2 = self.add_weight(name='value_weight2', shape=(self.out_shape, self.out_shape), 
                               initializer='glorot_uniform', trainable=True)
        self.V3 = self.add_weight(name='value_weight3', shape=(self.out_shape, self.out_shape), 
                               initializer='glorot_uniform', trainable=True)
        self.V4 = self.add_weight(name='value_weight4', shape=(self.out_shape, self.out_shape), 
                               initializer='glorot_uniform', trainable=True)
        #value bias
        # self.V_b = self.add_weight(name='value_bias', shape=(self.batch_size, self.out_shape), 
        #                        initializer='zeros', trainable=True)
        #output projection
        self.W = self.add_weight(name='output_weight', shape=(4*self.out_shape, self.out_shape), 
                               initializer='glorot_uniform', trainable=True)

    def call(self,x):
        keys = tf.matmul(x, self.K)
        keys1 = tf.matmul(keys, self.K1)
        keys2 = tf.matmul(keys, self.K2)
        keys3 = tf.matmul(keys, self.K3)
        keys4 = tf.matmul(keys, self.K4)
        queries = tf.matmul(x, self.Q)
        queries1 =  tf.matmul(queries, self.Q1)
        queries2 =  tf.matmul(queries, self.Q2)
        queries3 =  tf.matmul(queries, self.Q3)
        queries4 =  tf.matmul(queries, self.Q4)
        values = tf.matmul(x, self.V)
        values1 = tf.matmul(values, self.V1)
        values2 = tf.matmul(values, self.V2)
        values3 = tf.matmul(values, self.V3)
        values4 = tf.matmul(values, self.V4)

        #attention scores
        attn_scores1 = K.softmax(tf.matmul(queries1, keys1, transpose_b=True))
        attn_scores2 = K.softmax(tf.matmul(queries2, keys2, transpose_b=True))
        attn_scores3 = K.softmax(tf.matmul(queries3, keys3, transpose_b=True))
        attn_scores4 = K.softmax(tf.matmul(queries4, keys4, transpose_b=True))

        weighted_values1 = tf.matmul(attn_scores1, values1)
        weighted_values2 = tf.matmul(attn_scores2, values2)
        weighted_values3 = tf.matmul(attn_scores3, values3)
        weighted_values4 = tf.matmul(attn_scores4, values4)

        weighted_values_stacked = tf.concat([weighted_values1, weighted_values2, weighted_values3, weighted_values4], axis=1)

        weighted_values = tf.matmul(weighted_values_stacked, self.W)

        return weighted_values

def build_attention_model(size):
    input = tf.keras.layers.Input(shape=(size,), dtype='float32', batch_size=41)
    embedding_size = size//4
    a = tf.keras.layers.Dense(embedding_size)(input)
    a = tf.keras.layers.BatchNormalization()(a)
    b = attention(embedding_size)(a)
    b = tf.keras.layers.BatchNormalization()(b)
    c = tf.keras.layers.Dense(128)(b)
    c = tf.keras.layers.BatchNormalization()(c)
    d = tf.keras.layers.Dense(64)(c)
    d = tf.keras.layers.BatchNormalization()(d)
    out = tf.keras.layers.Dense(1)(d)
    model = tf.keras.Model(input, out)
    return model

def build_multiheadattention_model(size):
    input = tf.keras.layers.Input(shape=(size,), dtype='float32', batch_size=41)
    embedding_size = size//4
    a = tf.keras.layers.Dense(embedding_size)(input)
    a = tf.keras.layers.BatchNormalization()(a)
    b = multihead_attention(embedding_size)(a)
    b = tf.keras.layers.BatchNormalization()(b)
    c = tf.keras.layers.Dense(128)(b)
    c = tf.keras.layers.BatchNormalization()(c)
    d = tf.keras.layers.Dense(64)(c)
    d = tf.keras.layers.BatchNormalization()(d)
    out = tf.keras.layers.Dense(1)(d)
    model = tf.keras.Model(input, out)
    return model

In [1]:
import numpy as np

In [4]:
mae_per_fold_soap_se_nomordred_atten = np.load('mae_per_fold_soap_se_nomordred_atten.npy')
mae_per_fold_soap_se_nomordred_multih_atten = np.load('mae_per_fold_soap_se_nomordred_multih_atten.npy')
mae_per_fold_soap_se_noBDEs_atten = np.load('mae_per_fold_soap_se_noBDEs_atten.npy')
mae_per_fold_soap_se_noBDEs_multih_atten = np.load('mae_per_fold_soap_se_noBDEs_multih_atten.npy')
mae_per_fold_soap_DeltaH_atten = np.load('mae_per_fold_soap_DeltaH_atten.npy')
mae_per_fold_soap_DeltaH_multih_atten = np.load('mae_per_fold_soap_DeltaH_multih_atten.npy')
mae_per_fold_lmbtr_se_noBDEs_atten = np.load('mae_per_fold_lmbtr_se_noBDEs_atten.npy')
mae_per_fold_lmbtr_se_noBDEs_multih_atten = np.load('mae_per_fold_lmbtr_se_noBDEs_multih_atten.npy')
mae_per_fold_lmbtr_DeltaH_atten = np.load('mae_per_fold_lmbtr_DeltaH_atten.npy')
mae_per_fold_lmbtr_DeltaH_multih_atten = np.load('mae_per_fold_lmbtr_DeltaH_multih_atten.npy')

In [5]:
print(np.mean(mae_per_fold_soap_se_nomordred_atten), '+-', np.std(mae_per_fold_soap_se_nomordred_atten))
print(np.mean(mae_per_fold_soap_se_nomordred_multih_atten), '+-', np.std(mae_per_fold_soap_se_nomordred_multih_atten))
print(np.mean(mae_per_fold_soap_se_noBDEs_atten), '+-', np.std(mae_per_fold_soap_se_noBDEs_atten))
print(np.mean(mae_per_fold_soap_se_noBDEs_multih_atten), '+-', np.std(mae_per_fold_soap_se_noBDEs_multih_atten))
print(np.mean(mae_per_fold_soap_DeltaH_atten), '+-', np.std(mae_per_fold_soap_DeltaH_atten))
print(np.mean(mae_per_fold_soap_DeltaH_multih_atten), '+-', np.std(mae_per_fold_soap_DeltaH_multih_atten))
print(np.mean(mae_per_fold_lmbtr_se_noBDEs_atten), '+-', np.std(mae_per_fold_lmbtr_se_noBDEs_atten))
print(np.mean(mae_per_fold_lmbtr_se_noBDEs_multih_atten), '+-', np.std(mae_per_fold_lmbtr_se_noBDEs_multih_atten))
print(np.mean(mae_per_fold_lmbtr_DeltaH_atten), '+-', np.std(mae_per_fold_lmbtr_DeltaH_atten))
print(np.mean(mae_per_fold_lmbtr_DeltaH_multih_atten), '+-', np.std(mae_per_fold_lmbtr_DeltaH_multih_atten))

25.32825126647949 +- 0.5740777877160542
25.163867950439453 +- 0.5831178904417817
24.837464332580566 +- 0.5375252095028092
24.16331787109375 +- 0.4419022135626598
25.06620864868164 +- 0.6251650250267081
24.655581855773924 +- 0.6270337958096727
25.32667407989502 +- 0.6380985052940673
24.995582389831544 +- 0.4578610446590041
25.3678560256958 +- 0.6919343370797165
24.741838264465333 +- 0.6690168319894865


In [30]:
dot = Digraph(
    node_attr={'shape': 'box', 'fontname': 'Roboto Light'},
    graph_attr={'fontname': 'Roboto Light', 'splines': 'ortho', 'rankdir': 'LR'}
)

dot.attr('node', shape='box3d')
dot.node('A', 'Input Layer')
dot.attr('node', shape='box')
dot.node('B', 'Hidden Layer\nN/4 Nodes')
dot.node('C', 'Attention')
dot.node('D', 'Hidden Layer\n128 Nodes')
dot.node('E', 'Hidden Layer\n64 Nodes')
dot.attr('node', shape='box3d')
dot.node('F', 'Output Layer')
dot.attr('node', shape='box')

dot.edge('A', 'B')
dot.edge('B', 'C')
dot.edge('C', 'D')
dot.edge('D', 'E')
dot.edge('E', 'F')

dot.format = 'png'
dot.render('Atten_NN_general')

'Atten_NN_general.png'

# Find the optimal activation function

In [ ]:
def build_model4(input_shape, no_nodes, activation):
    input = tf.keras.layers.Input(shape=(input_shape), dtype='float32')
    out = tf.keras.layers.Dense(no_nodes[0], activation=activation)(input)
    out = tf.keras.layers.BatchNormalization()(out)
    out = tf.keras.layers.Dense(no_nodes[1], activation=activation)(out)
    out = tf.keras.layers.BatchNormalization()(out)
    out = tf.keras.layers.Dense(no_nodes[2], activation=activation)(out)
    out = tf.keras.layers.BatchNormalization()(out)
    out = tf.keras.layers.Dense(1)(out)
    model = tf.keras.Model(input, out)
    return model

def build_model5(input_shape, no_nodes, activation):
    input = tf.keras.layers.Input(shape=(input_shape), dtype='float32')
    out = tf.keras.layers.Dense(no_nodes[0], activation=activation)(input)
    out = tf.keras.layers.BatchNormalization()(out)
    out = tf.keras.layers.Dense(no_nodes[1], activation=activation)(out)
    out = tf.keras.layers.BatchNormalization()(out)
    out = tf.keras.layers.Dense(no_nodes[2], activation=activation)(out)
    out = tf.keras.layers.BatchNormalization()(out)
    out = tf.keras.layers.Dense(no_nodes[3], activation=activation)(out)
    out = tf.keras.layers.BatchNormalization()(out)
    out = tf.keras.layers.Dense(1)(out)
    model = tf.keras.Model(input, out)
    return model